# EDITO API Helper

This Notebook includes basic functions to interact with the EDITO API.

Login is required for most actions and so it is advised to do it first. Whenever an action fails due to lack of authorization, refresh the access token.

In [ ]:
!pip3 install requests

import requests
from xml.etree import ElementTree
import json
from time import sleep

In [ ]:
username = "my_edito_username"
password = "my_edito_password"
process_id = "observation-assimilation-iliad-pyopia-0.0.2"

processes_endpoint = "https://api.dive.edito.eu/processes"
minio_endpoint = "https://minio.dive.edito.eu"
token_endpoint = "https://auth.dive.edito.eu/auth/realms/datalab/protocol/openid-connect/token"

## Authentication

In [ ]:
def login(username: str, password: str):
    data = {
        "client_id": "onyxia",
        "username": username,
        "password": password,
        "grant_type": "password",
        "scope": "openid",
    }
    res = requests.post(token_endpoint, data=data)
    if res.status_code != 200:
        return None
    return res.json()


def refresh_token(token: str):
    data = {
        "client_id": "onyxia",
        "grant_type": "refresh_token",
        "refresh_token": token,
    }
    response = requests.post(token_endpoint, data=data)
    if response.status_code != 200:
        return None
    return response.json()


def auth_s3(username: str, password: str):
    headers = {"Content-Type": "application/x-www-form-urlencoded"}

    data = {
        "client_id": "onyxia-minio",
        "username": username,
        "password": password,
        "grant_type": "password",
        "scope": "openid email profile",
    }

    response = requests.post(token_endpoint, headers=headers, data=data)
    json_response = response.json()
    access_token = json_response["access_token"]

    params = {
        "Action": "AssumeRoleWithWebIdentity",
        "WebIdentityToken": access_token,
        "DurationSeconds": "86400",
        "Version": "2011-06-15",
    }

    response = requests.post(minio_endpoint, params=params)

    root = ElementTree.fromstring(response.content)
    namespace_as_text = root.tag[root.tag.find("{") + 1 : root.tag.find("}")]
    namespace = {"ns": namespace_as_text}
    access_key_id = root.find(".//ns:AccessKeyId", namespace).text
    secret_access_key = root.find(".//ns:SecretAccessKey", namespace).text
    session_token = root.find(".//ns:SessionToken", namespace).text

    return {
        "access_key_id": access_key_id,
        "secret_access_key": secret_access_key,
        "session_token": session_token,
    }

### Login

In [ ]:
if auth_data := login(username, password):
    print(f"Authenticated as {username}.")
else:
    print("Failed authentication.")

### Refresh token

In [ ]:
if auth_data := refresh_token(auth_data["refresh_token"]):
    print("Refreshed access token.")
else:
    print("Failed refreshing access token.")

### S3 credentials

In [ ]:
s3_credentials = auth_s3(username, password)

## Execution

### List Processes

List all processes in the EDITO platform. Output format is YAML and can be parsed with ease.

In [ ]:
def list_processes():
    res = requests.get(
        f"{processes_endpoint}/processes",
        headers={"accept": "application/json"},
    )
    if res.status_code != 200:
        return None
    return res.json()["processes"]

In [ ]:
print("processes:")
if procs := list_processes():
    for i in procs:
        print(f"- title: {i['title']}")
        print(f"  name: {i['id']}")
        print(f"  description: {i['description']}")

### Get Process by ID

Display all information of a Process found by its ID.

In [ ]:
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {auth_data['access_token']}",
}

r = requests.get(f"{processes_endpoint}/processes/{process_id}", headers=headers)
print(f"Process {process_id} details:")
print(json.dumps(r.json(), indent=2))

### Execute Process

Write Process input values and request its execution.

Inputs must follow schema provided.

If container images are private, credentials must be written to pullSecret.

In [ ]:
process_inputs = {
    "onyxia": {"friendlyName": process_id, "share": False},
    "inputs": {
        "dir_name": "pyopia_inputs",
        "directory": "https://pipe-drive.inesctec.pt/inputs/pyopia/"
    },
    "resources": {
        "limits": {"cpu": "4200m", "memory": "8Gi"},
        "requests": {"cpu": "1000m", "memory": "6Gi"},
    },
    "s3": {
        "enabled": True,
        "endpoint": minio_endpoint.replace("https://",""),
        "defaultRegion": "waw3-1",
        "accessKeyId": s3_credentials["access_key_id"],
        "secretAccessKey": s3_credentials["secret_access_key"],
        "sessionToken": s3_credentials["session_token"],
    },
}

In [ ]:
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {auth_data['access_token']}",
}

r = requests.post(
    f"{processes_endpoint}/processes/{process_id}/execution",
    data=json.dumps(
        {
            "processInputs": process_inputs,
            "metadata": {},
        }
    ),
    headers=headers,
)

assert r.status_code == 201 and r.reason == "Created", (
    f"ERROR: request failed: {r.status_code} {r.reason}"
)

job_tmp = r.json()
print(json.dumps(job_tmp, indent=2))
job_id = job_tmp["jobID"]

### Monitor Job status

Every 20s checks job status.

Automatically refreshes access token.

In [ ]:
status = "running"
wait_time = 20

endpoint = f"{processes_endpoint}/jobs/{job_id}"

while status == "running":
    sleep(wait_time)

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {auth_data['access_token']}",
    }

    r = requests.get(endpoint, headers=headers)

    if r.status_code == 401:
        print("Unauthorized access. Refreshing token...")
        auth_data = refresh_token(auth_data["refresh_token"])
        continue

    job_tmp = r.json()
    print(json.dumps(job_tmp))
    status = job_tmp["status"]


### Get Job results

Display all results of a job.

In [ ]:
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {auth_data['access_token']}",
}

r = requests.get(
    f"{processes_endpoint}/jobs/{job_id}/results",
    headers=headers,
)

results_tmp = r.json()
print(json.dumps(results_tmp, indent=2))
